In [1]:
edges = [
    ('A', 'r1', 'B'),
    # ('B', 'r1', 'C'),
    # ('X', 'r1', 'C'),
    # ('A', 'r1', 'X'),
    # ('Y', 'r1', 'B'),
]

In [2]:
import networkx as nx
import pandas as pd
import random

random.seed(10)

In [3]:
df = pd.DataFrame(edges, columns=["head", "relation", "tail"])

In [4]:
df.head()

,head,relation,tail
0,A,r1,B


In [5]:
G = nx.from_pandas_edgelist(df, source="head", target="tail", edge_attr="relation")

In [6]:
G.nodes()

NodeView(('A', 'B'))

In [7]:
G.edges()

EdgeView([('A', 'B')])

In [8]:
r1_score_list = []
for n1 in G.nodes():
    for n2 in G.nodes():
        r1_score_list.append((n1, n2, random.random()))

In [9]:
sorted(r1_score_list, key=lambda x: x[2], reverse=True)

[('B', 'A', 0.5780913011344704),
 ('A', 'A', 0.5714025946899135),
 ('A', 'B', 0.4288890546751146),
 ('B', 'B', 0.20609823213950174)]

In [10]:
g_edges = list(G.edges())

In [11]:
g_edges

[('A', 'B')]

In [12]:
print(('A', 'B') in g_edges)
print(('A', 'C') in g_edges)

True
False


In [13]:
from linkpred.metrics import hits_at_k, mean_rank, mean_reciprocal_rank

print(hits_at_k(r1_score_list, g_edges, 1))
print(hits_at_k(r1_score_list, g_edges, 3))
print(hits_at_k(r1_score_list, g_edges, 4))
print(mean_rank(r1_score_list, g_edges))
print(mean_reciprocal_rank(r1_score_list, g_edges))

(0.0, 0)
(0.3333333333333333, 1)
(0.25, 1)
3.0
0.3333333333333333
